In [31]:
import sklearn as sk
import pandas as pd

In [32]:
# First, lets construct the different models that we will use.
from sklearn import tree
from sklearn import neighbors
from sklearn import naive_bayes
from sklearn import svm
from sklearn import neural_network

# read data engineered csv
df = pd.read_csv('H-2B_Engineered_Data.csv')
labels = df.loc[:,'CASE_STATUS']
features = pd.DataFrame(df.drop(labels = ['CASE_STATUS'], axis = 1))

classifiers = []

#classifiers
norm_tree = tree.DecisionTreeClassifier()
classifiers.append(('tree', norm_tree))
depth_tree = tree.DecisionTreeClassifier(max_depth = 10)
classifiers.append(('depth_tree', depth_tree))
knn = neighbors.KNeighborsClassifier(n_neighbors = 5)
classifiers.append(('knn', knn))
nb = naive_bayes.GaussianNB()
classifiers.append(('nb', nb))
sv = svm.SVC(gamma = 'auto')
classifiers.append(('svm', sv))
net = neural_network.MLPClassifier()
classifiers.append(('neural_network', net))

In [9]:
features["NAICS_CODE"].value_counts()

56    2870
72    1152
23     769
71     705
11     203
31     122
33     103
32      91
54      73
44      68
48      66
42      64
21      57
81      49
61      48
53      42
45      14
22      12
62      12
55      11
51       3
49       2
92       2
52       2
Name: NAICS_CODE, dtype: int64

In [10]:
features["SOC_CODE"].value_counts()

37    3547
35     892
47     783
39     435
51     263
53     219
45     175
43      73
49      45
41      44
33      21
27      18
11      10
31       4
25       3
17       3
29       2
19       1
15       1
13       1
Name: SOC_CODE, dtype: int64

In [33]:
from sklearn import decomposition
from sklearn import preprocessing as pp
from sklearn import neighbors as knn
from sklearn import pipeline
from sklearn import model_selection
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


# Build a preprocessor to scale numeric features and one hot encode categorical features
numeric_features = ['NBR_WORKERS_REQUESTED', 
                    'BASIC_NUMBER_OF_HOURS', 
                    'BASIC_RATE_OF_PAY', 
                    'SUPERVISE_HOW_MANY', 
                    'NUM_OF_MONTHS_TRAINING',
                    'EMP_EXP_NUM_MONTHS',
                    'WORK_DAY_LENGTH']
numeric_transformer = pipeline.Pipeline(steps=[
    ('scaler', pp.StandardScaler())])

categorical_features = ['SOC_CODE',
                        'NAICS_CODE',
                       'NATURE_OF_TEMPORARY_NEED',
                       'EDUCATION_LEVEL',
                       'CITY_MATCH',
                       'STATE_MATCH']
categorical_transformer = pipeline.Pipeline(steps=[
    ('onehot', pp.OneHotEncoder(sparse = False, handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])



In [34]:
from sklearn import metrics

# The results from the individual classifiers

for name, cl in classifiers:
    pipe = pipeline.Pipeline(steps = [('preprocess', preprocessor), ('dim', decomposition.PCA()), ('classifier', cl)])
    pr = model_selection.cross_val_predict(pipe, features, labels, cv = 5)
    
    print(cl)
    print(metrics.confusion_matrix(labels, pr))
    print(metrics.classification_report(labels, pr))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
[[ 314  475]
 [ 584 5167]]
              precision    recall  f1-score   support

           0       0.35      0.40      0.37       789
           1       0.92      0.90      0.91      5751

    accuracy                           0.84      6540
   macro avg       0.63      0.65      0.64      6540
weighted avg       0.85      0.84      0.84      6540

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_sa

/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)
[[ 293  496]
 [ 204 5547]]
              precision    recall  f1-score   support

           0       0.59      0.37      0.46       789
           1       0.92      0.96      0.94      5751

    accuracy                           0.89      6540
   macro avg       0.75      0.67      0.70      6540
weighted avg       0.88      0.89      0.88      6540



/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [35]:
from sklearn import ensemble

voting = ensemble.VotingClassifier(classifiers)

pipe = pipeline.Pipeline(steps = [('preprocess', preprocessor), ('dim', decomposition.PCA()), ('classifier', voting)])
pr = model_selection.cross_val_predict(pipe, features, labels, cv = 5)
    
print(voting)
print(metrics.confusion_matrix(labels, pr))
print(metrics.classification_report(labels, pr))

/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/trent/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

VotingClassifier(estimators=[('tree',
                              DecisionTreeClassifier(class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features=None,
                                                     max_leaf_nodes=None,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=1,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
                                                     presort=False,
                                                     random_state=None,
                                                     splitter='best')),
    